In [1]:
import numpy as np
import pylab as pl
import os
import pandas as pd
import glob
from mvpa2.datasets.mri import fmri_dataset

from mvpa2.measures import rsa
from mvpa2.measures.searchlight import sphere_searchlight

/opt/anaconda3/envs/py27/lib/python2.7/site-packages/mvpa2/base/hdf5.py:40: H5pyDeprecationWarning: The h5py.highlevel module is deprecated, code should import directly from h5py, e.g. 'from h5py import File'.
  import h5py.highlevel  # >= 2.8.0, https://github.com/h5py/h5py/issues/1063
/opt/anaconda3/envs/py27/lib/python2.7/site-packages/mvpa2/testing/tools.py:81: DeprecationWarning: Importing from numpy.testing.decorators is deprecated since numpy 1.15.0, import from numpy.testing instead.
  from numpy.testing.decorators import skipif


In [2]:
# little helper function to plot dissimilarity matrices
# since we are using correlation-distance, we use colorbar range of [0,2]
def plot_mtx(mtx, labels, title):
    pl.figure()
    pl.imshow(mtx, interpolation='nearest')
    pl.xticks(range(len(mtx)), labels, rotation=-45)
    pl.yticks(range(len(mtx)), labels)
    pl.title(title)
    pl.clim((0, 2))
    pl.colorbar()

In [3]:
bids_dir = '/data/projects/social_doors/'
os.chdir(bids_dir)

outp_dir = os.path.join(bids_dir, 'derivatives', 'rsa_searchlight')
data_dir = os.path.join(bids_dir, 'derivatives','social_doors')

# Input subject list
subjs_scan_info = pd.read_csv(bids_dir+'/derivatives/participants_good.tsv', sep='\t')
subjs_list = list(subjs_scan_info['participant_id'].unique())

subjs_list.sort()
print('Found '+str(len(subjs_list))+' subjects')
subj = subjs_list[0]

# Find beta maps
file_list = glob.glob(os.path.join(data_dir, subj, 'beta*nii.gz'))
file_list.sort()
print('Found '+str(len(file_list))+' beta maps')




Found 68 subjects
Found 12 beta maps


Find conditions (stimuli for the study). In this case, the beta maps were numbered by referencing the order of stimuli for the first run of the first subject.

In [4]:
conditions = ['negative','negative_win','negative_loss',
             'positive','positive_win','positive_loss']


## searchlight consistency measure
how correlated are the structures across runs

In [5]:
dscm = rsa.PDistConsistency()
sl_cons = sphere_searchlight(dscm, 2)

# Hypothesis RDMs

In [6]:
def prep_hyp_rdm(raw_df):
    # Temporarily replace the zeros in this dataframe so that the lower triangle can be captured by filtering out zeros
    raw_df = raw_df.replace(0, 5)
    
    # Make the diagonal 0 again
    for i in raw_df.index:
        for j in raw_df.columns:
            if i == j:
                raw_df.loc[i, j] = 0


    # Create 1-D vectors for each component and 
    upper_rdm = np.triu(np.array(raw_df)).flatten()
    upper_rdm = upper_rdm[upper_rdm != 0]
    
    upper_rdm[upper_rdm == 5] = 0
    
    return(upper_rdm)

### Valence

In [7]:
valence_df = pd.DataFrame([[0,0,0,1,1,1],
                          [0,0,0,1,1,1],
                          [0,0,0,1,1,1],
                          [1,1,1,0,0,0],
                          [1,1,1,0,0,0],
                          [1,1,1,0,0,0]],
                         index=conditions, columns=conditions)

valence_upper = prep_hyp_rdm(valence_df)


len(valence_upper)

15

### Correct

In [8]:
correct_df = pd.DataFrame([[0,0,0,0,0,0],
                          [0,0,1,0,0,1],
                          [0,1,0,0,1,0],
                          [0,0,0,0,0,0],
                          [0,0,1,0,0,1],
                          [0,1,0,0,1,0]],
                         index=conditions, columns=conditions)

correct_upper = prep_hyp_rdm(correct_df)


crctval_df = pd.DataFrame([[0,0,0,0,0,0],
                          [0,1,1,0,1,1],
                          [0,1,0,0,1,0],
                          [0,0,0,0,0,0],
                          [0,1,1,0,1,1],
                          [0,1,0,0,1,0]],
                         index=conditions, columns=conditions)

crctval_upper = prep_hyp_rdm(crctval_df)

Combine component vectors into a matrix for the multiple regression. Each column is one component.

In [9]:
rdms_all = np.array([valence_upper, correct_upper, crctval_upper]).T
rdms_all

array([[0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 1, 1],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1]])

In [10]:

from mvpa2.base.learner import ChainLearner
from mvpa2.mappers.shape import TransposeMapper


In [11]:
# plot the spatial distribution using NiPy
import nibabel as nb


import nibabel as nib


# Searchlight RSA for All Subjects - Subject-level

In [12]:
# Define searchlight
tdsm = rsa.Regression(rdms_all, normalize=True)
sl_tdsm = sphere_searchlight(tdsm, 4)

In [13]:
rdms_all.shape[1]

3

In [14]:
rdm_names = ['valence', 'correct', 'crctval']

In [15]:
print("Running subject ...")
for subj in subjs_list:
    print("..."+subj)
    
    file_list = glob.glob(os.path.join(data_dir, subj, 'tmap*.nii.gz'))
    file_list.sort()
    
    for task in ['mdoors','social']:
        func_files = [x for x in file_list if 'tmap_'+task in x]
        func_files = [x for x in func_files if 'V' not in x]
        func_files.sort()

        if len(func_files) < 6:
            continue
        else:

            subj_mni_mask = bids_dir+"derivatives/fmriprep/"+subj+"/func/"+subj+"_task-"+task+"_run-1_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

            betas = fmri_dataset(func_files, mask=subj_mni_mask)
            betas.sa.targets = np.array(conditions)

            slres_tdsm = sl_tdsm(betas)

            for n in range(rdms_all.shape[1]):
                vol = betas.a.mapper.reverse1(slres_tdsm.samples[n])
                img = nib.Nifti1Image(vol, betas.a.imgaffine)
                nib.save(img, os.path.join(outp_dir, subj+'_sl_'+task+'_'+rdm_names[n]+'.nii.gz')) 
            

Running subject ...
...sub-010


/opt/anaconda3/envs/py27/lib/python2.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


 * Please note: warnings are printed only once, but underlying problem might occur many times *
...sub-013


KeyboardInterrupt: 

# Searchlight RSA for All Subjects - Group-level

In [ ]:
file_list = glob.glob(os.path.join(data_dir, 'group', 'cond*tmap.nii'))
file_list.sort()
#file_list = file_list[:-2]
    
subj_mni_mask = "derivatives/reliability_analysis/relscenarios/Anatomy/"+subj+"_space-MNI152NLin2009cAsym_label-GM_probseg_bin.nii.gz"
    
tmaps = fmri_dataset(file_list, mask=subj_mni_mask)
tmaps.sa.targets = np.array(conditions)
    
slres_tdsm_group = sl_tdsm(tmaps)
    
for n in range(1,6):
    vol = tmaps.a.mapper.reverse1(slres_tdsm_group.samples[n-1])
    img = nib.Nifti1Image(vol, tmaps.a.imgaffine)
    nib.save(img, os.path.join(output_dir, 'group_sl_comp'+str(n)+'.nii.gz'))

# Supplementary Analyses

## Original Theoretical Dimensions